# Simple RNN (Recurrent Neural Network)

In [1]:
import numpy as np
from keras.callbacks import (
    CSVLogger,
    EarlyStopping,
    ModelCheckpoint,
    ReduceLROnPlateau,
    TensorBoard,
)
from keras.layers import SimpleRNN, Dense, Input
from keras.losses import MeanSquaredError
from keras.metrics import MeanSquaredError as MSEMetric
from keras.models import Sequential
from keras.optimizers import Adam

In [2]:
# The Fibonacci Sequence
def fibonacci(n):
    """Generate Fibonacci sequence up to n terms."""
    fib_list = [0, 1]
    for i in range(2, n):
        fib_list.append(fib_list[-1] + fib_list[-2])
    return fib_list[:n]


raw_seq = fibonacci(15)
n_steps = 5  # Use 5 previous numbers to predict the next
n_features = 1


# Function to convert a sequence into (X, y) samples
def create_sequences(sequence, n_steps):
    X, y = [], []
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence) - 1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


X, y = create_sequences(raw_seq, n_steps)

# Reshape input to be [samples, timesteps, features] for LSTM
X = X.reshape((X.shape[0], X.shape[1], n_features))

print(f"Sample X (Input): {X[0].flatten()} -> y (Output): {y[0]}")
print(f"Sample X (Input): {X[-1].flatten()} -> y (Output): {y[-1]}")

Sample X (Input): [0 1 1 2 3] -> y (Output): 5
Sample X (Input): [ 34  55  89 144 233] -> y (Output): 377


In [3]:
simple_rnn = Sequential(
    layers=[
        Input(shape=(n_steps, n_features)),
        SimpleRNN(64),
        Dense(64),
        Dense(32),
        Dense(1),
    ],
    name="simple_rnn",
)

In [4]:
simple_rnn.layers

[<SimpleRNN name=simple_rnn, built=True>,
 <Dense name=dense, built=True>,
 <Dense name=dense_1, built=True>,
 <Dense name=dense_2, built=True>]

In [5]:
simple_rnn.summary()

Model: "simple_rnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 64)             │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,497 (41.00 KB)

 Trainable params: 10,497 (41.00 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
simple_rnn.compile(optimizer=Adam(), loss=MeanSquaredError(), metrics=[MSEMetric()])  # type: ignore

In [ ]:
early_stopping_callback = EarlyStopping(
    monitor="loss", patience=10, restore_best_weights=True
)
model_checkpoint_callback = ModelCheckpoint(
    filepath="../../Models/simple_rnn.keras",
    monitor="loss",
    save_best_only=True,
    mode="min",
)
tensorboard_callback = TensorBoard(
    log_dir="../../Logs/simple_rnn_logs",
    histogram_freq=1,
    write_images=True,
    write_steps_per_second=True,
)
# tensorboard --logdir="Logs/simple_rnn_logs"

csvlogger_callback = CSVLogger("../../Logs/simple_rnn_logs.csv")

reduce_lr_callback = ReduceLROnPlateau(monitor="loss", factor=0.5, patience=5)

simple_rnn_history = simple_rnn.fit(
    X,
    y,
    epochs=200,
    callbacks=[
        early_stopping_callback,
        model_checkpoint_callback,
        tensorboard_callback,
        csvlogger_callback,
        reduce_lr_callback,
    ],
)

Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 22916.2031 - mean_squared_error: 22916.2031 - learning_rate: 0.0010
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - loss: 22712.3711 - mean_squared_error: 22712.3711 - learning_rate: 0.0010
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - loss: 22510.8340 - mean_squared_error: 22510.8340 - learning_rate: 0.0010
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step - loss: 22310.8496 - mean_squared_error: 22310.8496 - learning_rate: 0.0010
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 539ms/step - loss: 22111.7324 - mean_squared_error: 22111.7324 - learning_rate: 0.0010
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - loss: 21912.7344 - mean_squared_error: 21912.7344 - learning_rate: 0.0010
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - loss: 21713.0547 - mean_squared_error: 21713.0547 - learning_rate: 0.0010
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step - loss: 21511.9316 - mean_squared_error: 21511.9316 - learning_

In [8]:
# New input data (the last n_steps numbers)
x_input = np.array([55, 89, 144, 233, 377])

# Reshape the input for the model: (1 sample, n_steps timesteps, 1 feature)
x_input = x_input.reshape((1, n_steps, n_features))

# Make the prediction
yhat = simple_rnn.predict(x_input)

print(f"\n--- Prediction Result ---")
print(f"Input Sequence: {x_input.flatten()}")
# Round the prediction as Fibonacci numbers are integers
print(f"Predicted Next Number: {round(yhat[0][0])}")
print(f"Expected Next Number: ", x_input.flatten()[-1] + x_input.flatten()[-2])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step

--- Prediction Result ---
Input Sequence: [ 55  89 144 233 377]
Predicted Next Number: 381
Expected Next Number:  610
